# Data visualisation

In [14]:
import torch
import json
import os

In [3]:
root = '/DATATWO/users/mincut/Object-Centric-VideoAnswering'
datadir='/DATATWO/users/mincut/Object-Centric-VideoAnswering/data/train' 

In [4]:
f=json.load(open(os.path.join(datadir,'train.json')))

In [5]:
categories = dict()
for pt in f:
    key = pt["question_type"]
    if key in categories.keys():
        categories[key] += 1
    else:
        categories[key] = 1
print(categories)

KeyError: 'question_type'

In [ ]:
type(f[0])

In [ ]:
f[0].keys()

In [25]:
f[1]['scene_index']

1

In [26]:
print(f[1]['video_filename'])

video_00001.mp4


In [31]:
for x in ((f[1]['questions'])):
    print(x['question'],x['question_type'])

What is the color of the object that exits the scene? descriptive
What material is the object to collide with the red object? descriptive
What is the material of the object to collide with the cube? descriptive
How many gray spheres exit the scene after the cylinder enters the scene? descriptive
How many rubber objects exit the scene? descriptive
Are there any cylinders that exit the scene before the gray sphere exits the scene? descriptive
What color is the stationary rubber object when the video begins? descriptive
What material is the moving sphere when the video ends? descriptive
How many moving metal objects are there when the video ends? descriptive
How many moving cylinders are there? descriptive
Are there any moving metal objects when the video ends? descriptive
Which of the following is responsible for the gray object's exit? explanatory
Which of the following is not responsible for the purple sphere's colliding with the gray object? explanatory
What will happen next? predicti

In [30]:
%%HTML
<video width="320" height="240" controls>
  <source src="video_00000.mp4" type="video/mp4">
</video>

In [9]:
import os, json 
root='/DATATWO/users/mincut/Object-Centric-VideoAnswering/data'
questions_path = os.path.join(root,"train",f"train.json")
dtst = json.load(open(questions_path))
print(len(dtst),dtst[0]['questions'][0].keys())

10000 dict_keys(['question_id', 'question', 'question_type', 'question_subtype', 'program', 'answer'])


In [16]:
root='/DATATWO/users/mincut/Object-Centric-VideoAnswering/data'
for mode in ['train','validation','test']:
    questions_path = os.path.join(root,mode,f"{mode}.json")
    cat = dict()
    dtst = json.load(open(questions_path))
    for dt_pt in dtst:
        for q in dt_pt['questions']:
            import pdb ; pdb.set_trace()
            cat[q['question_type']] = cat.get(q['question_type'],0)+1
            print(q['question_id'])
    print(cat)

> /tmp/ipykernel_4140473/2163411507.py(9)<module>()
      7         for q in dt_pt['questions']:
      8             import pdb ; pdb.set_trace()
----> 9             cat[q['question_type']] = cat.get(q['question_type'],0)+1
     10             print(q['question_id'])
     11     print(cat)

ipdb> print(q.keys())
dict_keys(['question_id', 'question', 'question_type', 'question_subtype', 'program', 'answer'])
ipdb> dt_pt.keys()
dict_keys(['questions', 'scene_index', 'video_filename'])
ipdb> dt_pt['video_filename']
'video_00000.mp4'
ipdb> exit 


# Preprocessing, Dataset and DataLoader 

In [46]:
##def preprocess
import cv2
import os

# Open the video file

def preprocess_video(path):

    cap = cv2.VideoCapture(path)

    # Create a directory to store the extracted frames
    name = path.split('/')[-1]
    name = name.split('.')[0].split('_')[-1]
    mode = path.split('/')[-4]
    
    output_dir = f'extracted_frames/{mode}/{name}'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Set the time interval to extract frames (in milliseconds)
    interval = 250

    # Get the frame rate of the video
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Calculate the number of frames to skip
    skip_frames = int(fps * (interval / 1000))

    # Initialize variables
    frame_count = 0
    success = True

    # Loop through the video frames
    while success:
        # Read a frame from the video
        success, frame = cap.read()
        # Check if the frame was read successfully
        if success:
            # Check if the current frame is a multiple of the skip frames
            if frame_count % skip_frames == 0:
                # Save the extracted frame as an image file
                output_path = os.path.join(output_dir, f"frame{frame_count}.jpg")
                cv2.imwrite(output_path, frame)
                

            # Increment the frame count
            frame_count += 1

    # Release the video capture object and close all windows
    cap.release()
    cv2.destroyAllWindows()


In [47]:
# preprocess_video('data/train/video/video_00000-01000/video_00000.mp4')
import glob
path = 'data/'
videos = [f for f in glob.glob(path + "**/*.mp4", recursive=True)]
for video in videos:
    preprocess_video(video)

## Dataset

In [35]:
data = '/DATATWO/users/mincut/Object-Centric-VideoAnswering/data'
import os, sys
import json
import torch
from torch.utils.data import Dataset
import cv2

In [38]:
class VidQA(Dataset):
    
    def __init__(self,mode='train'):
        
        self.dtst_fldr = os.path.join(data,mode)
        self.mode = mode
        self.ques_file = os.path.join(self.dtst_fldr,f"{mode}.json")
        self.ques,self.ques_types,self.ques_vid = self.read_questions()
#         self.frames = self.read_frames()
    
    def read_frames(self,filename):
        folder = os.path.join(root,'extracted_frames',mode)
        idx = filename.split('_')[-1]
        frame_fldr = os.path.join(folder,idx)
        frm_seq = []
        for fl in os.scandir(frame_fldr):
            img = cv2.imread(fl.path)
            # Convert the image from BGR to RGB format
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # Convert image to torch tensor
            img = torch.from_numpy(img.transpose((2, 0, 1)))
            frm_seq.append(img)
                        
        return torch.stack(frm_seq)
        
    def read_questions(self):
        mode = self.mode 
        q_path = os.path.join(root,mode,f"{mode}.json")
        cat = dict()
        q_list = []
        qt_list = []
        qt_vid = []
        dtst = json.load(open(q_path))
        for dt_pt in dtst:
            vid_idx = dt_pt['video_filename'].split('.')[0].split('_')[-1]
            vid_idx = int(vid_idx)
            for q in dt_pt['questions']:
                cat[q['question_type']] = cat.get(q['question_type'],0)+1
                q_list.append(q['question'])
                qt_list.append(q['question_id'])
                qt_vid.append(vid_idx)
                
        return q_list,qt_list,qt_vid 
    
    def gen_vocab(self):
        pass 
        
    def tokenize_sentences(self):
        pass 
        
    
    def __len__(self):
        return len(self.ques)
    
    def __getitem__(self,idx):
        
        return frm 
        
    

In [39]:
trn_dtst = VidQA()